# Clases. Conceptos avanzados
---

En un cuaderno previo, introdujimos el concepto de clase. Vamos a seguir explorando conceptos mñas avanzados de la programación orientada a objetos en Kotlin como la herencia, polimorfismo, clases abstractas y modificadores de acceso

## 1. Herencia
---

Como ya sabemos, la idea principal tras la herencia es la de propagar una serie de atributos y comportamientos comunes hacia abajo por una jerarquía de clases.

<br>Al igual que en Java, una clase en Kotlin sólo podrá heredar de **único** padre (además de la clase **```Any```**, similar a **Object** en Java, de la que heredan todas)

In [ ]:
data class Nota(val letra: Char, val puntos: Double, val creditos: Double)

open class Persona(var nombre: String, var apellidos: String) {
    fun nombreCompleto() = "$nombre $apellidos"
}

class Estudiante(nombre: String, apellidos: String,
    var notas: MutableList<Nota> = mutableListOf())
    : Persona(nombre, apellidos) {

    fun addNota(nota: Nota) {
        notas += nota
    }
}

En el ejemplo anterior, la clase **```Estudiante```** **hereda** de **```Persona```**. Veamos algunas particularidades respecto a la sintaxis:

- Para indicar que una clase es subclase de otra (hereda), lo indicamos añadiendo dos puntos (**:**) y el nombre de la superclase antes del cuerpo de la misma
- Para poder heredar de una clase, ésta tiene que tener el modificador **```open```**. E Kotlin, las clases y métodos son finales por defecto, por lo que no se pueden heredar unas ni sobreescribir los otros
- En Estudiante, los parámetros **```nombre```** y **```apellidos```** son simples argumentos (fíjate que no usamos ```var``` ni ```val```) que empleamos para pasar al constructor primario de **```Persona```**, que es donde están definidas dichas **propiedades** y que heredará **```Estudiante```**
- **```Estudiante```** también hereda cualquier método definido en **```Persona```**, pero sólo lo podrá sobreescribir si está declarado como **```open```**

In [ ]:
val john = Persona(nombre = "John", apellidos = "Doe")
val jane = Estudiante(nombre = "Jane", apellidos = "Doe")

println(john.nombreCompleto())
println(jane.nombreCompleto())

val mates1 = Nota(letra = 'B', puntos = 8.5, creditos = 3.0)
jane.addNota(mates1)

### Polimorfismo

Una de las características de la programación orientada a objetos, es la posibilidad de tratar de forma diferente un objeto en base al contexto. Es lo que llamamos **polimorfismo**

<br>El polimorfismo está intimamente ligado con la herencia. El hecho de que una variable de un tipo de una superclase pueda referenciar a cualquier objeto de una subclase, nos permite tratar de forma conjunta a todas esas instancias hijas. Al mismo tiempo, gracias al polimorfismo, se invocará en cada caso la implementación específica de los métodos heredados y sobreescritos.

In [ ]:
open class Animal(val nombre: String) {
    open fun habla() { print("Soy $nombre: ") }
}

class Gato(mote: String): Animal(mote) {
    override fun habla() { 
        super.habla()
        println("miau!") 
    }
}

class Perro(mote: String): Animal(mote) {
    override fun habla() { 
        super.habla()
        println("guau!") 
    }
}

val animales = listOf<Animal>(Gato("Silvestre"), Perro("Scooby"), Gato("Tom"))
animales.forEach { it.habla() }

Respecto al ejemplo anterior:

- El método **```habla()```** del padre debe estar definido como **```open```** para poder sobreescribirlo por las clases hijas
- A su vez, las clases hijas lo "marcarán" como **```override```** para indicar que están sobreescribiendo un método del padre
- Vemos como la lista está definnida para contener objetos de tipo **```Animal```**, aunque las instancias añadidas son todas de alguno de sus hijos
- Cada vez que invoquemos el método ```**habla()**``` de alguna de estas instancias contenidas en la lista, se invocará la versión específica del objeto concreto (**polimorfismo**)
- Desde una clase hija podemos acceder a los miembros del padre haciendo uso de la referencia **```super```**


### Chequeo de Tipo

Debido al polimorfismo, nos podemos encontrar en situaciones donde necesitemos verificar el tipo específico de la instancia referenciada por una variable. Para ello, podemos usar el operador **```is```**

In [ ]:
// constante de tipo Animal que referencia una instancia de Gato
val animal: Animal = Gato("Tom")

println(animal is Gato)
println(animal is Perro)

También disponemos de operadores que nos van a permitir hacer un **cast** de forma segura hacia un supertipo o un subtipo

- **```as```**, nos permite hacer un **unsafe cast** en tiempo de compilación a un tipo que sabemos que no va a fallar (por ejemplo, a un supertipo)
- **```as?```**, nos permite hacer un **safe cast** que, en caso de fallar, nos devolverá **```null```** (por ejemplo, a un subtipo)

In [ ]:
val tom = Gato("Tom") // es un Gato

val animal = tom as Animal 
val perro = animal as? Perro // este cast devolverá nulo

println(perro)

### Sobreescritura de Métodos

Las subclases, además de heredar las propiedades y métodos de su superclase y definir los suyos propios, puede **sobreescribir** los métodos heredados de su padre.

<br>Como ya se comentó previamente, el método de la superclase debe ir acompañado del modificador **```open```**. De forma similar, el método sobreescrito en la sublase irá precedido por **```override```**, resaltando de forma inequívoca que estamos sobreescribiendo un método heredado. Lógicamente, el método sobreescrito deberá tener la misma firma que el método del padre

In [ ]:
open class Shape {
    open fun area(): Double { return 0.0  }
}

class Rect(val width: Double, val height: Double): Shape() {
    override fun area() = width * height
}

class Circle(val radius: Double): Shape() {
    override fun area() = kotlin.math.PI * radius * radius
}

In [ ]:
val r = Rect(2.0, 4.0)
println("area del rectángulo = ${r.area()}")

val c = Circle(2.0)
println("area del círculo = ${c.area()}")

### Clases abstractas

En determinadas situaciones, como las clases de ejemplo **```Animal```** o **```Shape```**, no tiene sentido instanciar directamente objetos de las mismas. Podemos prevenir esto definiéndolas como **clases abstractas**. Para ello, antepondremos el modificador **```abstract```** en la declaración de la clase.

<br>Al contrario de lo que ocurre con una clase normal, las clases abstractas son **```open```** por defecto (lo contrario no tendría sentido)

<br>Dentro de la clase abstracta podemos tener métodos con cuerpo y métodos sin él. Estos últimos deberán declarase a su vez como **```abstract```** y las subclases estarán obligadas a implementarlos (o a ser declaradas como clases abstractas)

<br>Por ejemplo, vamos a redefinir nuestra clase **```Shape```** anterior como clase abstracta:

In [ ]:
abstract class Shape {
    abstract fun area(): Double // no tiene cuerpo
}

class Rect(val width: Double, val height: Double): Shape() {
    override fun area() = width * height
}

class Circle(val radius: Double): Shape() {
    override fun area() = kotlin.math.PI * radius * radius
}

In [ ]:
// no podemos crear objetos de Shape
// la siguiente línea generaría un error
//val geom = Shape()

val r = Rect(2.0, 4.0)
println("area del rectángulo = ${r.area()}")

val c = Circle(2.0)
println("area del círculo = ${c.area()}")